In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow import keras

In [ ]:
TPS_file_path = '../input/tabular-playground-series-jan-2021/train.csv'
TPS_data = pd.read_csv(TPS_file_path)
TPS_data.shape

In [ ]:
TPS_data.head()

In [ ]:
TPS_data.drop('id', axis=1, inplace=True)
TPS_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
# mieszamy nasze dane i dzielimy na zbiory treningowy i walidacyjny

train_set, valid_set = train_test_split(TPS_data, test_size=0.2, random_state=42, shuffle=True)

[train_set.shape, valid_set.shape]

In [ ]:
X_train = train_set.drop('target', axis=1)
X_valid = valid_set.drop('target', axis=1)

y_train = train_set.target
y_valid = valid_set.target

In [ ]:
X_train.hist(figsize=(20,15), bins=50)

In [ ]:
X_train.hist(column='cont1', bins=50)

In [ ]:
X_train.max().sort_values(ascending=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler #normalizacja danych

cols = X_train.columns

scaler = MinMaxScaler(feature_range=(0,1)) #sieć neuronowa 'lubi' dane w zakresie od 0 do 1

X_train = pd.DataFrame(scaler.fit_transform(X_train)) #scaler uczymy na X_train
X_valid = pd.DataFrame(scaler.transform(X_valid)) #normalizujemy na podstawie statystyk z X_train

In [ ]:
X_train.columns = cols
X_train.head()

In [ ]:
X_train.hist(column='cont1', bins=50)

In [ ]:
X_train.max().sort_values(ascending=False)

# Podstawowy model

In [ ]:
from keras import layers

model = keras.models.Sequential([
    layers.InputLayer(input_shape=[14]),
    layers.Dense(50, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(1)
])

In [ ]:
model.compile(loss='mse', #funkcję straty rmse wprowadzimy w modelu zaawansowanym
              optimizer=keras.optimizers.SGD(lr=0.01),
              #metrics=['mse']
             )

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

In [ ]:
TEST_data_filepath = '../input/tabular-playground-series-jan-2021/test.csv'
TEST_data = pd.read_csv(TEST_data_filepath)

TEST_data.columns

In [ ]:
id = TEST_data.id

In [ ]:
TEST_data = TEST_data.drop('id', axis=1)
TEST_data = scaler.transform(TEST_data)

target = model.predict(TEST_data).flatten()
target

In [ ]:
output = pd.DataFrame({'id': id,
                      'target': target})
output.to_csv('submission.csv', index=False)